In [1]:
import numpy as np
import torch
from datasets import load_dataset
from transformers import AutoTokenizer
from torch.utils.data import Dataset
import logging

from datasets import load_dataset

raw_datasets  = load_dataset("glue", 'sst2')

In [2]:
from datasets import DatasetDict, Dataset


def generate_prompt(data_point):
    #print(data_point)
    # sorry about the formatting disaster gotta move fast
    return f"""Below is an input that describes a task. Classify the sentiment of this input as positive or negative.
                ### Input: {data_point}
                ### Response: """ # noqa: E501


# Assuming 'dataset' is your original DatasetDict object
def add_textual_sentiment(example):
    if example['label'] == 1:
        example['output'] = 'positive'
    else:
        example['output'] = 'negative'
    example['input'] = generate_prompt(example['sentence']) 
    example['labels'] = example['label'] 
    return example

# Apply the function to add the 'output' column
test_data = raw_datasets['validation'].map(add_textual_sentiment)

# Now you can inspect your dataset
print(test_data)UCF


Dataset({
    features: ['sentence', 'label', 'idx', 'output', 'input', 'labels'],
    num_rows: 872
})


In [3]:
train_data = raw_datasets['train'].map(add_textual_sentiment)

In [4]:

from transformers import AutoTokenizer, AutoModelForMaskedLM, AutoConfig
#from roberta import RobertaForSequenceClassification

from transformers import AutoTokenizer, AutoModelForCausalLM, AutoModelForSequenceClassification
from huggingface_hub import login


# Log in using your Hugging Face token
login("yourtoken")

model_name = "HuggingFaceTB/SmolLM-135M-Instruct" 

tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token



The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: read).
Your token has been saved to /home/kowsher/.cache/huggingface/token
Login successful


In [5]:
from generator.modeling import PredictorCausalLM
from generator.collator import DataCollator
from generator import metrics
from generator.training import GenTrainer

/home/kowsher/.local/lib/python3.10/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: '/home/kowsher/.local/lib/python3.10/site-packages/torchvision/image.so: undefined symbol: _ZN3c1017RegisterOperatorsD1Ev'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(


In [6]:
config = AutoConfig.from_pretrained(model_name)  # Load configuration
config.dense_representation = 10 
config.pad_token_id = tokenizer.pad_token_id
config.nub_of_token_generation = 2

model = PredictorCausalLM(config, num_labels=2) 

In [7]:
import leader

leader.PEFT(model, method='column', rank=3)

In [8]:
data_collator = DataCollator(tokenizer=tokenizer)

In [9]:
from transformers import TrainingArguments, Trainer

import time
from transformers import Trainer, TrainingArguments
training_args = TrainingArguments(
    output_dir='dir',
    learning_rate=2e-4,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    gradient_accumulation_steps=4,
    num_train_epochs=50,
    weight_decay=0.00,
    remove_unused_columns=False,
    evaluation_strategy="steps",
    save_strategy="steps",
    save_total_limit=2,
    save_steps=10000000,
    logging_steps=200,

    load_best_model_at_end=True,
    lr_scheduler_type="cosine",  # You can choose from 'linear', 'cosine', 'cosine_with_restarts', 'polynomial', etc.
    warmup_steps=200,
)

trainer = GenTrainer(
    model=model,
    args=training_args,
    train_dataset=train_data,
    eval_dataset=test_data,
    data_collator=data_collator,
    compute_metrics=metrics.classification_metrics,
    max_steps_for_sampling=200,

)


/home/kowsher/miniconda3/envs/LD/lib/python3.10/site-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [ ]:
trainer.train()

[2024-12-23 22:23:45,947] [INFO] [real_accelerator.py:203:get_accelerator] Setting ds_accelerator to cuda (auto detect)


/home/kowsher/miniconda3/envs/LD/compiler_compat/ld: cannot find -laio: No such file or directory
collect2: error: ld returned 1 exit status
/home/kowsher/miniconda3/envs/LD/compiler_compat/ld: cannot find -laio: No such file or directory
collect2: error: ld returned 1 exit status
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend. Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: ahmedshuvo969 (projectstevens). Use `wandb login --relogin` to force relogin


Step,Training Loss,Validation Loss,Precision,Recall,F1-score,Accuracy
200,6.158700,5.088578,0.611648,0.590027,0.571495,0.594037
400,4.084900,4.586310,0.827184,0.826303,0.825629,0.825688
600,3.831700,4.427312,0.865849,0.865970,0.865817,0.865826
800,3.751400,4.344441,0.882103,0.878852,0.879206,0.879587
1000,3.704900,4.272298,0.881911,0.880231,0.880507,0.880734
1200,3.633800,4.244632,0.896787,0.896712,0.896745,0.896789
1400,3.586000,4.270648,0.880804,0.866454,0.866602,0.868119
1600,3.559500,4.201682,0.903636,0.903764,0.903657,0.903670
1800,3.564800,4.182253,0.903672,0.903595,0.903629,0.903670
2000,3.536100,4.173313,0.903877,0.903469,0.903597,0.903670


We detected that you are passing `past_key_values` as a tuple and this is deprecated and will be removed in v4.43. Please use an appropriate `Cache` class (https://huggingface.co/docs/transformers/v4.41.3/en/internal/generation_utils#transformers.Cache)
